[메이저 리그 야구선수 연봉 데이터] Hitters 데이터는 다음과 같이 ISLR 패키지를 설치한 후 얻을 수 있다. 그런데 이 데이터에는 결측치가 존재하기 때문에 na.omit() 함수를 사용하여 결측치를 제외하기로 한다. 실제 분석에 사용할 데이터는 263명의 야구선수에 대한 20가지의 기록으로 구성되어 있다.

In [3]:
library(ISLR)
Hitters = na.omit(Hitters)
dim(Hitters)

Warning message:
"패키지 'ISLR'는 R 버전 4.3.3에서 작성되었습니다"


[1] 263  20

Salary를 반응변수로 나머지 19개의 변수를 설명변수로 하는 회귀모형을 가정하고 다음 질문에 답하여라.


### 문제 1
PRESS(prediction sum of squares), Mallow's Cp, AIC, BIC 기준 최적의 모형을 전진선택법, 후진제거법, 전체탐색법(all subset selection) 방법을 이용하여 찾아보아라. 해당 기준들에 의하여 선택된 모형을 비교하여 보고, 본인의 최적 모형을 적절한 근거와 함께 제시하여라.

In [35]:
library(leaps)
library(MASS)

In [63]:
# 1-1. PRESS 기준 전진선택법

# PRESS 계산해주는 함수 cal_press 정의
cal_press = function(model) {
    res = residuals(model)
    hat = lm.influence(model)$hat
    press = sum((res / (1-hat))^2)
    return(press)
}

# 선택된 변수를 저장하는 변수. 처음에는 전진선택을 하기 전 상태이므로 비어있다.
selected_vars = c()
# 종속변수
response_var = "Salary"
# 단계별 최적 PRESS 비교를 위한 변수. 최초에는 무한대값을 입력
prev_best_press <- Inf

for (i in 1:(ncol(Hitters)-1 )) {
    # 후보변수는 Hitters 컬럼에서 종속변수와 이미 선택된 변수를 제외한 변수들이다.
    available_vars = setdiff(names(Hitters), c(response_var, selected_vars))

    # 이번 단계의 최적 PRESS와 그에 따른 독립변수를 저장할 변수
    best_press = Inf
    best_var = NULL

    # 각각의 후보변수에 대해서 적합모델을 구하고, 적합모델로 PRESS를 계산한다.
    for (var in available_vars) {
        formula = as.formula(paste(response_var, "~", paste(c(selected_vars, var), collapse="+")))
        model = glm(formula, data=Hitters)
        press = cal_press(model)
        # cat("선택된 변수", var, "\tPress 값:", press, "\n")
        if(press < best_press) {
            best_press = press
            best_var = var
        }
    }
    # 이전 루프에서의 best_press 값과 비교하여 변수 선택 중단 여부 결정
    if (best_press >= prev_best_press) {
        cat(i, "단계에서 선택된 변수의 PRESS 값이 이전 루프의 값보다 큽니다. 변수 선택을 중단합니다.\n\n")
        break
    }

    selected_vars = c(selected_vars, best_var)
    cat(i, "단계: 변수", best_var,"가 선택되었습니다. \t PRESS는", best_press, "입니다. \n")

    # 현재의 best_press 값을 이전 루프에서의 값으로 업데이트
    prev_best_press <- best_press
}

cat("선택된 변수들:", paste(selected_vars, collapse = "+"), "\n")
cat("최종 선택된 모델의 press 값은", prev_best_press, "\n")

1 단계: 변수 CRBI 가 선택되었습니다. 	 PRESS는 37203615 입니다. 
2 단계: 변수 Hits 가 선택되었습니다. 	 PRESS는 31968283 입니다. 
3 단계: 변수 Division 가 선택되었습니다. 	 PRESS는 30832958 입니다. 
4 단계: 변수 PutOuts 가 선택되었습니다. 	 PRESS는 29784686 입니다. 
5 단계: 변수 AtBat 가 선택되었습니다. 	 PRESS는 29146016 입니다. 
6 단계: 변수 Walks 가 선택되었습니다. 	 PRESS는 28389013 입니다. 
7 단계에서 선택된 변수의 PRESS 값이 이전 루프의 값보다 큽니다. 변수 선택을 중단합니다.

선택된 변수들: CRBI+Hits+Division+PutOuts+AtBat+Walks 
최종 선택된 모델의 press 값은 28389013 


In [72]:
# 최적모델식
model = glm(Salary ~ AtBat + Hits + Walks + CRBI + Division + PutOuts, data=Hitters)
summary(model)

cat("적합된 모델은 다음과 같다.\n")
cat("Salary = 91.512 - 1.869 * AtBat + 7.60 * Hits + 3.698 * Walks + 0.643 * CRBI - 122.952 * DivisionW + 0.264 * PutOuts\n")


Call:
glm(formula = Salary ~ AtBat + Hits + Walks + CRBI + Division + 
    PutOuts, data = Hitters)

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)   91.51180   65.00006   1.408 0.160382    
AtBat         -1.86859    0.52742  -3.543 0.000470 ***
Hits           7.60440    1.66254   4.574 7.46e-06 ***
Walks          3.69765    1.21036   3.055 0.002488 ** 
CRBI           0.64302    0.06443   9.979  < 2e-16 ***
DivisionW   -122.95153   39.82029  -3.088 0.002239 ** 
PutOuts        0.26431    0.07477   3.535 0.000484 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 102323.8)

    Null deviance: 53319113  on 262  degrees of freedom
Residual deviance: 26194904  on 256  degrees of freedom
AIC: 3789.2

Number of Fisher Scoring iterations: 2


적합된 모델은 다음과 같다.
Salary = 91.512 - 1.869 * AtBat + 7.60 * Hits + 3.698 * Walks + 0.643 * CRBI - 122.952 * DivisionW + 0.264 * PutOuts


In [73]:
# 1-2. Mallow's Cp 기준 전진선택법
fw = regsubsets(Salary ~ ., data=Hitters, method="forward")

# 모델 요약
sum_fw = summary(fw)

# Mallow's Cp 값들 확인
cp_values = sum_fw$cp
print(cp_values)

# Cp 값이 가장 작은 것은 8번째 단계이다.
cat("Cp 값이 가장 작은 것은", which.min(cp_values), "번째 단계이다.\n")

# Mallow's Cp 기준으로 최적모델 선택
args = sum_fw$which[which.min(cp_values),]

# 최적모델의 변수 확인
cat("\nMallow's Cp 기준으로 선택된 변수들:\n")
vars = names(args[args])
print(vars)

[1] 104.281319  50.723090  38.693127  27.856220  21.613011  14.023870  13.607126
[8]   7.624674


Cp 값이 가장 작은 것은 8 번째 단계이다.

Mallow's Cp 기준으로 선택된 변수들:
[1] "(Intercept)" "AtBat"       "Hits"        "Walks"       "CRuns"      
[6] "CRBI"        "CWalks"      "DivisionW"   "PutOuts"    


In [74]:
# 최적모델식
model = glm(Salary ~ AtBat + Hits + Walks + CRuns + CRBI + CWalks + Division + PutOuts, data=Hitters)
summary(model)

cat("적합된 모델은 다음과 같다.\n")
cat("Salary = 117.152 - 2.034 * AtBat + 6.855 * Hits + 6.441 * Walks + 0.705 * CRuns + 0.527 * CRBI - 0.807 * CWalks - 123.780 * DisivisonW + 0.275 * Putouts\n")


Call:
glm(formula = Salary ~ AtBat + Hits + Walks + CRuns + CRBI + 
    CWalks + Division + PutOuts, data = Hitters)

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  117.15204   65.07016   1.800 0.072985 .  
AtBat         -2.03392    0.52282  -3.890 0.000128 ***
Hits           6.85491    1.65215   4.149 4.56e-05 ***
Walks          6.44066    1.52212   4.231 3.25e-05 ***
CRuns          0.70454    0.24869   2.833 0.004981 ** 
CRBI           0.52732    0.18861   2.796 0.005572 ** 
CWalks        -0.80661    0.26395  -3.056 0.002483 ** 
DivisionW   -123.77984   39.28749  -3.151 0.001824 ** 
PutOuts        0.27539    0.07431   3.706 0.000259 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 99052.1)

    Null deviance: 53319113  on 262  degrees of freedom
Residual deviance: 25159234  on 254  degrees of freedom
AIC: 3782.6

Number of Fisher Scoring iterations: 2


적합된 모델은 다음과 같다.
Salary = 117.152 - 2.034 * AtBat + 6.855 * Hits + 6.441 * Walks + 0.705 * CRuns + 0.527 * CRBI - 0.807 * CWalks - 123.780 * DisivisonW + 0.275 * Putouts


In [80]:
# 1-3. AIC 기준 전진선택법

# 모든 변수를 포함하는 모델 정의
full_model = glm(Salary ~ ., data=Hitters)

# 변수 없는 모델 정의
null_model = glm(Salary ~ 1, data=Hitters)

# stepAIC 함수를 사용하여 전진선택법 실시
model = stepAIC(null_model, scope = list(lower=null_model, upper=full_model), direction="forward")

selected_vars = names(coef(model))
cat("선택된 변수들:", paste(selected_vars, collapse="+"),"\n")

Start:  AIC=3964.13
Salary ~ 1

            Df Deviance    AIC
+ CRBI       1 36179679 3864.1
+ CRuns      1 36437951 3866.0
+ CHits      1 37253973 3871.8
+ CAtBat     1 38559403 3880.9
+ CHmRun     1 38626920 3881.4
+ CWalks     1 40526491 3894.0
+ RBI        1 42548030 3906.8
+ Walks      1 42814280 3908.4
+ Hits       1 43058621 3909.9
+ Runs       1 43919955 3915.1
+ Years      1 44760007 3920.1
+ AtBat      1 45009644 3921.6
+ HmRun      1 47045145 3933.2
+ PutOuts    1 48505013 3941.2
+ Division   1 51343011 3956.2
<none>         53319113 3964.1
+ Assists    1 53284615 3966.0
+ League     1 53308237 3966.1
+ Errors     1 53317558 3966.1
+ NewLeague  1 53318684 3966.1

Step:  AIC=3864.14
Salary ~ CRBI

            Df Deviance    AIC
+ Hits       1 30646560 3822.5
+ Runs       1 31003147 3825.5
+ Walks      1 31979946 3833.7
+ AtBat      1 32115095 3834.8
+ RBI        1 32871407 3840.9
+ PutOuts    1 32912644 3841.2
+ Division   1 34445793 3853.2
+ Years      1 34512340 3853.7
+ H

In [86]:
summary(model)

cat("적합된 모델은 다음과 같다.\n")
cat("Salary = 162.535 - 2.169 * AtBat + 6.912 * Hits + 5.773 * Walks - 0.130 * CAtBat + 1.408 * CRuns + 0.774 * CRBI - 0.831 * CWalks - 112.380 * DivisioinW + 0.297 * PutOuts + 0.283 * Assists\n")


Call:
glm(formula = Salary ~ CRBI + Hits + PutOuts + Division + AtBat + 
    Walks + CWalks + CRuns + CAtBat + Assists, data = Hitters)

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  162.53544   66.90784   2.429 0.015830 *  
CRBI           0.77431    0.20961   3.694 0.000271 ***
Hits           6.91802    1.64665   4.201 3.69e-05 ***
PutOuts        0.29737    0.07444   3.995 8.50e-05 ***
DivisionW   -112.38006   39.21438  -2.866 0.004511 ** 
AtBat         -2.16865    0.53630  -4.044 7.00e-05 ***
Walks          5.77322    1.58483   3.643 0.000327 ***
CWalks        -0.83083    0.26359  -3.152 0.001818 ** 
CRuns          1.40825    0.39040   3.607 0.000373 ***
CAtBat        -0.13008    0.05550  -2.344 0.019858 *  
Assists        0.28317    0.15766   1.796 0.073673 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 97223.82)

    Null deviance: 53319113  on 262  degrees of freedom

적합된 모델은 다음과 같다.
Salary = 162.535 - 2.169 * AtBat + 6.912 * Hits + 5.773 * Walks - 0.130 * CAtBat + 1.408 * CRuns + 0.774 * CRBI - 0.831 * CWalks - 112.380 * DivisioinW + 0.297 * PutOuts + 0.283 * Assists


In [87]:
# 1-4. BIC 기준 전진선택법

n = nrow(Hitters)

# stepAIC 함수를 사용하여 전진선택법 실시
model = stepAIC(null_model, scope = list(lower=null_model, upper=full_model), direction="forward", k=log(n))

selected_vars = names(coef(model))
cat("선택된 변수들:", paste(selected_vars, collapse="+"),"\n")

Start:  AIC=3967.7
Salary ~ 1

            Df Deviance    AIC
+ CRBI       1 36179679 3871.3
+ CRuns      1 36437951 3873.2
+ CHits      1 37253973 3879.0
+ CAtBat     1 38559403 3888.0
+ CHmRun     1 38626920 3888.5
+ CWalks     1 40526491 3901.1
+ RBI        1 42548030 3913.9
+ Walks      1 42814280 3915.6
+ Hits       1 43058621 3917.1
+ Runs       1 43919955 3922.3
+ Years      1 44760007 3927.3
+ AtBat      1 45009644 3928.7
+ HmRun      1 47045145 3940.4
+ PutOuts    1 48505013 3948.4
+ Division   1 51343011 3963.3
<none>         53319113 3967.7
+ Assists    1 53284615 3973.1
+ League     1 53308237 3973.2
+ Errors     1 53317558 3973.3
+ NewLeague  1 53318684 3973.3

Step:  AIC=3871.28
Salary ~ CRBI

            Df Deviance    AIC
+ Hits       1 30646560 3833.2
+ Runs       1 31003147 3836.2
+ Walks      1 31979946 3844.4
+ AtBat      1 32115095 3845.5
+ RBI        1 32871407 3851.6
+ PutOuts    1 32912644 3852.0
+ Division   1 34445793 3863.9
+ Years      1 34512340 3864.4
+ Hm

In [89]:
summary(model)
Hitters
cat("적합된 모델은 다음과 같다.\n")
cat("Salary = 91.512 - 1.869 * AtBat + 7.604 * Hits + 3.698 * Walks + 0.643 * CRBI - 122.952 * DivisionW \n")


Call:
glm(formula = Salary ~ CRBI + Hits + PutOuts + Division + AtBat + 
    Walks, data = Hitters)

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)   91.51180   65.00006   1.408 0.160382    
CRBI           0.64302    0.06443   9.979  < 2e-16 ***
Hits           7.60440    1.66254   4.574 7.46e-06 ***
PutOuts        0.26431    0.07477   3.535 0.000484 ***
DivisionW   -122.95153   39.82029  -3.088 0.002239 ** 
AtBat         -1.86859    0.52742  -3.543 0.000470 ***
Walks          3.69765    1.21036   3.055 0.002488 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 102323.8)

    Null deviance: 53319113  on 262  degrees of freedom
Residual deviance: 26194904  on 256  degrees of freedom
AIC: 3789.2

Number of Fisher Scoring iterations: 2


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<fct>
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.000,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.000,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.000,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.500,N
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.000,A
-Al Newman,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.000,A
-Argenis Salazar,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.000,A
-Andres Thomas,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.000,N
-Andre Thornton,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.000,A


적합된 모델은 다음과 같다.
Salary = 162.535 - 2.169 * AtBat + 6.912 * Hits + 5.773 * Walks - 0.130 * CAtBat + 1.408 * CRuns + 0.774 * CRBI - 0.831 * CWalks - 112.380 * DivisioinW + 0.297 * PutOuts + 0.283 * Assists


In [34]:
cat("\n참고로, 전진선택법 단계별로 선택된 변수는 다음과 같다.\n")
for(i in 1:8) {
    args = sum_fw$which[i,]
    vars = names(args[args])
    print(paste(vars, collapse=" + "))    
}

sum_fw$which


참고로, 전진선택법 단계별로 선택된 변수는 다음과 같다.
[1] "(Intercept) + CRBI"
[1] "(Intercept) + Hits + CRBI"
[1] "(Intercept) + Hits + CRBI + PutOuts"
[1] "(Intercept) + Hits + CRBI + DivisionW + PutOuts"
[1] "(Intercept) + AtBat + Hits + CRBI + DivisionW + PutOuts"
[1] "(Intercept) + AtBat + Hits + Walks + CRBI + DivisionW + PutOuts"
[1] "(Intercept) + AtBat + Hits + Walks + CRBI + CWalks + DivisionW + PutOuts"
[1] "(Intercept) + AtBat + Hits + Walks + CRuns + CRBI + CWalks + DivisionW + PutOuts"


,(Intercept),AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,LeagueN,DivisionW,PutOuts,Assists,Errors,NewLeagueN
1,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
2,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
3,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
4,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
5,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
6,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
7,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
8,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE


In [102]:
# RSS 기준으로 전진선택법 수행하기
fw = regsubsets(Salary ~ ., data=Hitters, method="forward.rss")

# 모델 요약
sum_fw = summary(fw)

# Mallow's Cp 값들 확인
RSS_values = sum_fw$rss
print(RSS_values)
cat("RSS 값이 가장 작은 것은", which.min(RSS_values), "번째 단계이다.\n")
# RSS 값이 가장 작은 것은 8번째 단계이다.

# RSS 기준으로 최적모델 선택
sum_fw$which

optimal_model = sum_fw$which[which.min(RSS_values),]

# 최적모델의 변수 확인
cat("RSS 기준으로 선택된 변수들:\n")
vars = names(optimal_model[optimal_model])
print(vars)

ERROR: Error in regsubsets.formula(Salary ~ ., data = Hitters, method = "forward.rss"): Ambiguous or unrecognised method name : forward.rss


In [5]:
Hitters

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<fct>
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.000,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.000,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.000,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.500,N
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.000,A
-Al Newman,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.000,A
-Argenis Salazar,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.000,A
-Andres Thomas,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.000,N
-Andre Thornton,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.000,A


In [ ]:
response_var = "Salary"
predictor_vars = setdiff(names(Hitters), response_var)

press_values = sapply(predictor_vars, function(var) {
    formula = as.formula(paste(response_var, "~", var))
    model = glm(formula, data=Hitters)
    press = cal_press(model)
    return(press)
})

best_var = predictor_vars[which.min(press_values)]
best_press = min(press_values)

cat("PRESS 값이 가장 작은 변수는", best_var, "\n")
cat("PRESS 값은", best_press, "\n\n")

print(press_values)

In [ ]:
selected_vars = c("CRBI")
predictor_vars = setdiff(names(Hitters), c(response_var, selected_vars))

press_values = sapply(predictor_vars, function(var) {
    formula = as.formula(paste(response_var, "~", paste(c(selected_vars, var), collapse = "+" )))
    model = glm(formula, data=Hitters)
    press = cal_press(model)
    return(press)
})

best_var = predictor_vars[which.min(press_values)]
best_press = min(press_values)

selected_vars <- c(selected_vars, best_var)

# 결과 출력
cat("두 번째 변수로 선택된 변수는", best_var, "\n")
cat("선택된 모델의 PRESS 값은", best_press, "\n")
cat("현재 선택된 변수들", paste(selected_vars, collapse=","), "\n")

print(press_values)

2. 1번 문제에서 선택한 모형이 (1)정규성을 따르는지 (2)오차항의 등분산성 가정을 만족하는지 검증해보아라. 또한 (3)이상점(outlier)이나 영향점(influential data)이 있는지 조사하여 보아라.

3.
  - AtBat
  - Hits
  - Walks
  - CAtBat
  - CRuns
  - CRBi
  - CWalks
  - League
  - Division
  - PutOuts
  - Assists  
를 설명변수로 하는 다중회귀를 적합시킨다고 할 때, 이 모형에 대한 2번 문제의 물음에 답해보아라.

[1] 263  20

In [3]:
Hitters

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<fct>
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.000,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.000,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.000,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.500,N
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.000,A
-Al Newman,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.000,A
-Argenis Salazar,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.000,A
-Andres Thomas,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.000,N
-Andre Thornton,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.000,A
